## Create tri-colored PNGs for all slices in HDF5s

In [25]:
import os, sys, re, psycopg2, subprocess
import numpy as np
from subprocess import call

#### Set context

In [6]:
stack_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines"
channel_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/"

os.chdir(stack_folder)
cwd = os.getcwd()
print("Current working directory %s" % cwd)

Current working directory /Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines


#### Get stack names

In [21]:
stacks = sorted([d for d in os.listdir(stack_folder) if not d.startswith('.')])
print(len(stacks), stacks)

186 ['6.7FRhcrtR-Gal4-uasKaede', 'Anti-5HT', 'Anti-Gad67', 'Anti-GlyR', 'Anti-TH', 'Anti-Zn1', 'Anti-Zn12(Hnk-1)', 'Anti-Znp1(Synaptotagmin2)', 'Anti-Zrf1(GFAP)', 'Anti-Zrf2', 'Anti-tERK', 'Elavl3-GCaMP5G', 'Elavl3-H2BRFP', 'EtVmat2-GFP', 'Gad1b-GFP', 'Glyt2-GFP', 'Hcrt-RFP', 'Isl1-GFP', 'Isl2bGal4-uasDendra', 'MH_13319-Gal4', 'MH_15576-Gal4', 'MH_16659-Gal4', 'MH_16715-Gal4', 'MH_16913-Gal4', 'MH_16914-Gal4', 'MH_16915-Gal4', 'MH_16916-Gal4', 'MH_16946-Gal4', 'MH_17056-Gal4', 'MH_17767-Gal4', 'MH_17768-Gal4', 'MH_17776-Gal4', 'MH_17777-Gal4', 'MH_17787-Gal4', 'MH_17845-Gal4', 'MH_17846-Gal4', 'MH_18107-Gal4', 'MH_18108-Gal4', 'MH_18327-Gal4', 'MH_18329-Gal4', 'MH_19322-Gal4', 'MH_19566-Gal4', 'MH_20360-Gal4', 'MH_20361-Gal4', 'MH_20454-Gal4', 'MH_20552-Gal4', 'MH_20553-Gal4', 'MH_20638-Gal4', 'MH_20639-Gal4', 'MH_20640-Gal4', 'MH_20641-Gal4', 'MH_20642-Gal4', 'MH_20643-Gal4', 'MH_20644-Gal4', 'MH_20646-Gal4', 'MH_20647-Gal4', 'MH_20648-Gal4', 'MH_20649-Gal4', 'MH_20652-Gal4', 'MH_2065

#### Create color channel folders & color subfolders

In [22]:
colors = ['red', 'blue', 'green', 'cyan']

for stack in stacks:
    folder_path = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/%s/" % stack
    try:
        os.mkdir(folder_path)
    except:
        pass
    for color in colors:
        channel_path = folder_path + color
        try:
            os.mkdir(channel_path)
        except:
            pass

#### Create color channel overlays for each stack

In [23]:
new_stacks = [s for s in stacks if 'MH_' in s]
for i, stack in enumerate(new_stacks):
    src = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/%s/" % stack
    base = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/%s/" % stack

    os.chdir(src)
    print(i, base)
    for color in colors:
        dest = base + color
        cwd = os.getcwd()
        print(cwd)
        try:
            subprocess.check_output(["mogrify", "-path", dest, "-format", "png", "+level-colors", "black,%s"%color, "-fuzz", "9%", "-transparent", "black", "*.jpg"], stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            print(err.output)


0 /Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/MH_13319-Gal4/
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_13319-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_13319-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_13319-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_13319-Gal4
1 /Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/MH_15576-Gal4/
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_15576-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_15576-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_15576-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_15576-Gal4
2 /Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/MH_16659-Gal4/
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_16659-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_16659-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/MH_16659-Gal4
/

In [31]:
# prepare postgreSQL table

create_table_command = (
"""
    DROP TABLE colorchannels;
    CREATE TABLE colorChannels (
        channel_id SERIAL PRIMARY KEY,
        channel_color TEXT NOT NULL,
        channel_name TEXT NOT NULL,
        channel_image_path TEXT NOT NULL
    );
""")

In [32]:
# connect to zbrain2db postgreSQL database

conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [33]:
# create table colorChannels

try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [34]:
from natsort import natsorted

In [35]:
channel_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/"
stack_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/" # stacks temporarily stored in folder (0-Lines)
stacks = sorted([d for d in os.listdir(stack_folder) if not d.startswith('.')])
print(stacks)

['6.7FRhcrtR-Gal4-uasKaede', 'Anti-5HT', 'Anti-Gad67', 'Anti-GlyR', 'Anti-TH', 'Anti-Zn1', 'Anti-Zn12(Hnk-1)', 'Anti-Znp1(Synaptotagmin2)', 'Anti-Zrf1(GFAP)', 'Anti-Zrf2', 'Anti-tERK', 'Elavl3-GCaMP5G', 'Elavl3-H2BRFP', 'EtVmat2-GFP', 'Gad1b-GFP', 'Glyt2-GFP', 'Hcrt-RFP', 'Isl1-GFP', 'Isl2bGal4-uasDendra', 'MH_13319-Gal4', 'MH_15576-Gal4', 'MH_16659-Gal4', 'MH_16715-Gal4', 'MH_16913-Gal4', 'MH_16914-Gal4', 'MH_16915-Gal4', 'MH_16916-Gal4', 'MH_16946-Gal4', 'MH_17056-Gal4', 'MH_17767-Gal4', 'MH_17768-Gal4', 'MH_17776-Gal4', 'MH_17777-Gal4', 'MH_17787-Gal4', 'MH_17845-Gal4', 'MH_17846-Gal4', 'MH_18107-Gal4', 'MH_18108-Gal4', 'MH_18327-Gal4', 'MH_18329-Gal4', 'MH_19322-Gal4', 'MH_19566-Gal4', 'MH_20360-Gal4', 'MH_20361-Gal4', 'MH_20454-Gal4', 'MH_20552-Gal4', 'MH_20553-Gal4', 'MH_20638-Gal4', 'MH_20639-Gal4', 'MH_20640-Gal4', 'MH_20641-Gal4', 'MH_20642-Gal4', 'MH_20643-Gal4', 'MH_20644-Gal4', 'MH_20646-Gal4', 'MH_20647-Gal4', 'MH_20648-Gal4', 'MH_20649-Gal4', 'MH_20652-Gal4', 'MH_20653-Ga

In [37]:
# create array of table insert commands for each color channel image

def create_insert_commands(stacks):
    channel_id = 0
    for stack in stacks:
        src = "/Users/OGN/1Projects/ZBrain2/app/assets/images/0-Lines/%s/" % stack
        base = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/%s/" % stack
        rel = "images/3-ColorChannels/%s/" % stack
    
        colors = ['red', 'blue', 'green', 'cyan']
        for color in colors:
            channel_folder = base + color
            channels = os.listdir(channel_folder)
            channels = [ "%s%s/%s" % (rel, color, channel) for channel in channels]
            channels = natsorted(channels)
            for channel_image_path in channels:
                insert_command = ("INSERT INTO colorChannels (channel_id, channel_color, channel_name, channel_image_path) values (%i, '%s', '%s', '%s')" 
                                 % (channel_id, color, stack, channel_image_path))
                channel_id += 1
                yield insert_command
insert_commands = create_insert_commands(stacks)

In [39]:
# use insert command array to create table entries for each color channel image

conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

for insert_command in insert_commands:
    try:
        cursor.execute(insert_command)
    except Exception as e:
        print(e)
        
cursor.close()
conn.commit()

In [43]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM colorChannels ORDER BY channel_id')

rows = cursor.fetchall()
for row in rows[14000:14100]:
    print(row)

(14000, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-60.png')
(14001, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-61.png')
(14002, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-62.png')
(14003, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-63.png')
(14004, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-64.png')
(14005, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-65.png')
(14006, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-66.png')
(14007, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-67.png')
(14008, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-68.png')
(14009, 'blue', 'MH_16915-Gal4', 'images/3-ColorChannels/MH_16915-Gal4/blue/MH_16915-Gal4-69.png')
(14010, 'b